In [1]:
import pandas as pd

In [2]:
# Load the data
games_df = pd.read_csv('compiledData/cleanedCompiledData.csv')
games_df.head()
X_q1 = games_df.filter(regex='q1$')
y = games_df['points']


In [3]:
# more imports lol
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error


# Trying MLP with Backpropagation on q1 stats only


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_q1, y, test_size=0.2, random_state=42)
regressor = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=5000, random_state=42)
def fit_and_score():
    regressor.fit(X_train, y_train)
    print(f"train score: {regressor.score(X_train, y_train)}")
    print(f"train mae: {mean_absolute_error(y_train, regressor.predict(X_train))}")
    print(f"test score: {regressor.score(X_test, y_test)}")
    print(f"test mae: {mean_absolute_error(y_test, regressor.predict(X_test))}")
fit_and_score()

train score: 0.3069176115326163
train mae: 6.816735484985063
test score: 0.049434879446500046
test mae: 8.16347117597591


### didn't think that would work superwell anyway lol

### Trying MLP with Backpropagation on q1 and q2 stats

In [5]:
# Select columns that end with 'q1' or 'q2'
q1_q2_columns = games_df.filter(regex='q[12]$')
X_train, X_test, y_train, y_test = train_test_split(q1_q2_columns, y, test_size=0.2, random_state=42)
fit_and_score()


train score: 0.5592765827093807
train mae: 5.367232848532716
test score: 0.20434348283446369
test mae: 7.430772020243554


### It's not as bad as it looks!!

true the accuracies are low, but the mae is almost within a touchdown, and predicting from only first half stats. I think that is really cool by itself.

### adding third quarter

In [6]:
# Select columns that end with 'q1' or 'q2' or q3
not_q4_columns = games_df.filter(regex='q[123]$')
X_train, X_test, y_train, y_test = train_test_split(not_q4_columns, y, test_size=0.2, random_state=42)
fit_and_score()

train score: 0.9233325856116259
train mae: 2.1450026743585227
test score: 0.025484062076488012
test mae: 8.203173711777039


### we acheived overfit 

### Trying to fix overfitting with some grid search (thanks copiilot)
#### I'll try with first half stats again

In [7]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Split the data
X_train, X_test, y_train, y_test = train_test_split(q1_q2_columns, y, test_size=0.2, random_state=42)

def grid_search():
    # Create a pipeline with scaling and MLPRegressor
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('mlp', MLPRegressor(max_iter=5000, random_state=42, early_stopping=True))
    ])
    
    # Define the parameter grid for GridSearchCV
    param_grid = {
        'mlp__hidden_layer_sizes': [(50, 50), (100, 100), (100, 50, 100)],
        'mlp__alpha': [0.0001, 0.001, 0.01],  # Regularization parameter
    }
    
    # Perform GridSearchCV to find the best parameters
    grid_searcher = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error')
    grid_searcher.fit(X_train, y_train)
    
    # Print the best parameters and the corresponding score
    print(f"Best parameters: {grid_searcher.best_params_}")
    print(f"Best cross-validation MAE: {-grid_searcher.best_score_}")
    
    # Evaluate the model on the test set
    best_model = grid_searcher.best_estimator_
    y_pred = best_model.predict(X_test)
    print(f"Test MAE: {mean_absolute_error(y_test, y_pred)}")
grid_search()

Best parameters: {'mlp__alpha': 0.01, 'mlp__hidden_layer_sizes': (100, 100)}
Best cross-validation MAE: 7.122288475116731
Test MAE: 7.139627512700424


### slight improvement :/
### I'll try with first three quarters

In [8]:
X_train, X_test, y_train, y_test = train_test_split(not_q4_columns, y, test_size=0.2, random_state=42)
grid_search()

Best parameters: {'mlp__alpha': 0.01, 'mlp__hidden_layer_sizes': (100, 50, 100)}
Best cross-validation MAE: 6.425600415453528
Test MAE: 6.497690201117577


### within +-7 points without any fourth quarter stats, honestly somewhat practical, however wouldn't say much in a close game. On the other hand professional sports analytics can't say much on a close game either, just look at the win prediction graphs for an average football game